<a href="https://colab.research.google.com/github/GrzegorzSzczepanek/deep_learning_fun/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report, roc_curve, roc_auc_score
import torch
from torch import nn

In [88]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [89]:
dataset = pd.read_csv("/content/drive/MyDrive/neuron_workplace/autentykacja_eeg.csv", delimiter=";")
dataset.head()

,Delta,Theta,Alpha_low,Alpha_high,Beta_low,Beta_high,Gamma_low,Gamma_high,Attention,Meditation,BlinkStrength,Flag
0,785389,114375,57106,37625,24464,17104,15846,8024,45,91,15,1
1,386949,80792,23506,22118,9566,7147,5392,1697,8,42,6,0
2,21453,14085,4756,5062,2176,2117,2867,862,48,66,9,1
3,322637,56936,17810,25793,15870,8050,3621,2100,25,69,8,0
4,19412,12985,6416,6131,7126,7419,2677,1884,80,53,-1,1


In [ ]:
X = torch.tensor(dataset.loc[:, "Delta": "BlinkStrength"].values, dtype=torch.float32)
y = torch.tensor(dataset.Flag.values, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [98]:
# len(X.columns.values)

# 1. Najpierw spróbuję napisać MLP bez dokonywania zmian w moich danych.
W moim datasecie nie ma pustych krotek. Na razie pominę standaryzację, by porównać efekty potem

In [99]:
X, y

(tensor([[ 7.8539e+05,  1.1438e+05,  5.7106e+04,  ...,  4.5000e+01,
           9.1000e+01,  1.5000e+01],
         [ 3.8695e+05,  8.0792e+04,  2.3506e+04,  ...,  8.0000e+00,
           4.2000e+01,  6.0000e+00],
         [ 2.1453e+04,  1.4085e+04,  4.7560e+03,  ...,  4.8000e+01,
           6.6000e+01,  9.0000e+00],
         ...,
         [ 8.0033e+04,  3.2374e+04,  7.0020e+03,  ...,  6.1000e+01,
           5.3000e+01,  1.5000e+01],
         [ 1.6945e+05,  3.8482e+04,  1.2956e+04,  ...,  6.2000e+01,
           8.7000e+01,  1.6000e+01],
         [ 1.0141e+05,  4.1010e+04,  1.0225e+04,  ...,  4.8000e+01,
           3.7000e+01, -1.0000e+00]]),
 tensor([1., 0., 1.,  ..., 0., 0., 1.]))

In [109]:
input = len(dataset.columns.values) - 1
output = len(y.unique())


class FlagModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.Linear(hidden_units, output_shape),
            nn.ReLU()
        )


    def forward(self, x):
        return self.block_1(x)


flag_model_1 = FlagModel(input, 10, output).to(device)
flag_model_1

FlagModel(
  (block_1): Sequential(
    (0): Linear(in_features=11, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=2, bias=True)
    (2): ReLU()
  )
)

In [110]:
def accuracy_fn(y_true: torch.Tensor, y_pred: torch.Tensor) -> [torch.float]:
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(params=flag_model_1.parameters(), lr=0.005)

EPOCHS = 5

In [111]:
def train_step(model: torch.nn.Module,
               data: torch.tensor,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    train_loss, train_acc = 0, 0
    model.train()

    y_logits = flag_model_1(data)
    y_pred = torch.round(torch.sigmoid(y_logits))

    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    return loss, acc



In [112]:
def test_step(model: torch.nn.Module,
               data: torch.tensor,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    model.eval()
    with torch.inference_mode():
        test_logits = flag_model

In [113]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

train_time_start = timer()


def train(model: torch.nn.Module,
          train_data: torch.tensor,
          test_data: torch.tensor,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          epochs: int,
          device:torch.device = device):

    for epoch in tqdm(range(EPOCHS)):
        train_step(mode=model,
                   data=train_data,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   accuracy_fn=accuracy_fn)
        test_step(mode=model,
                   data=test_data,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   accuracy_fn=accuracy_fn)

        train_time_end = timer()
        print(f"{(train_time_end - train_time_start):.3f}")

In [ ]:
#   from timeit import default_timer as timer
#   from tqdm.auto import tqdm
#   train_time_start = timer()

#   for epoch in tqdm(range(epochs)):

#     train_step(model=model,
#                dataloader=train_dataloader,
#                loss_fn=loss_fn,
#                optimizer=optimizer,
#                accuracy_fn=accuracy_fn)
#     validation_step(model=model,
#               dataloader=val_dataloader,
#               loss_fn=loss_fn,
#               accuracy_fn=accuracy_fn)
#     print(f"Epochs: {epoch}\n-----------------------")

#   train_time_end = timer()
#   print(f"{(train_time_end - train_time_start):.3f}")

In [ ]:
# def train_step(model: torch.nn.Module,
#                dataloader: torch.utils.data.DataLoader,
#                loss_fn: torch.nn.Module,
#                optimizer: torch.optim.Optimizer,
#                accuracy_fn,
#                device:torch.device = device):
#   train_loss, train_acc = 0, 0
#   model.train()
#   model.to(device)
#   for batch, (X,y) in enumerate(dataloader):
#     X, y = X.to(device), y.to(device)
#     y_pred = model(X)
#     loss = loss_fn(y_pred, y)
#     train_loss += loss
#     train_acc += accuracy_fn(y_true=y,
#                              y_pred=y_pred.argmax(dim=1))
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#   train_loss /= len(dataloader)
#   train_acc /= len(dataloader)
#   print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

In [ ]:
# def validation_step(model: torch.nn.Module,
#               dataloader: torch.utils.data.DataLoader,
#               loss_fn: torch.nn.Module,
#               accuracy_fn,
#               device: torch.device = device):
#   test_loss, test_acc = 0, 0
#   model.to(device)
#   model.eval()

#   with torch.inference_mode():
#     for X, y in dataloader:
#       X, y = X.float().to(device), y.to(device)

#       test_pred = model(X)
#       test_loss += loss_fn(test_pred, y)
#       test_acc += accuracy_fn(y_true=y,
#                               y_pred=test_pred.argmax(dim=1)
#       )

#     test_loss /= len(dataloader)
#     test_acc /= len(dataloader)
#     print(f"Validation loss: {test_loss:.5f} | Validation accuracy: {test_acc:.2f}%\n")

In [ ]:
# def train(model: torch.nn.Module,
#           train_dataloader: torch.utils.data.DataLoader,
#           val_dataloader: torch.utils.data.DataLoader,
#           loss_fn: torch.nn.Module,
#           optimizer: torch.optim.Optimizer,
#           accuracy_fn,
#           epochs: int,
#           device:torch.device = device):

#   from timeit import default_timer as timer
#   from tqdm.auto import tqdm
#   train_time_start = timer()

#   for epoch in tqdm(range(epochs)):

#     train_step(model=model,
#                dataloader=train_dataloader,
#                loss_fn=loss_fn,
#                optimizer=optimizer,
#                accuracy_fn=accuracy_fn)
#     validation_step(model=model,
#               dataloader=val_dataloader,
#               loss_fn=loss_fn,
#               accuracy_fn=accuracy_fn)
#     print(f"Epochs: {epoch}\n-----------------------")

#   train_time_end = timer()
#   print(f"{(train_time_end - train_time_start):.3f}")